In [36]:
# 原始的patch
# https://github.com/shuwang127/BinGNN/commit/137af4376381dec53bda56e6d01658ba9f3f2c55?diff=split
# 重新生成的0-context patch
# diff -0 -u  a/BinGNN.py b/BinGNN.py >p/BinGNN.py.patch

In [37]:
import numpy as np
import re

In [38]:
# read graph of A.
lines = open('a/BinGNN.py', 'r').readlines()
nodesA = np.array([['nodeidA'+str(linenum+1), 'BinGNN.py', 'funcname', linenum+1, lines[linenum]] for linenum in range(len(lines))], dtype=object)
# edgesA = [[.., .., .., ..] ..]
print(nodesA)

[['nodeidA1' 'BinGNN.py' 'funcname' 1 'import os\n']
 ['nodeidA2' 'BinGNN.py' 'funcname' 2 'import sys\n']
 ['nodeidA3' 'BinGNN.py' 'funcname' 3 'import time\n']
 ['nodeidA4' 'BinGNN.py' 'funcname' 4 'import argparse\n']
 ['nodeidA5' 'BinGNN.py' 'funcname' 5 'import numpy as np\n']
 ['nodeidA6' 'BinGNN.py' 'funcname' 6 'import torch\n']
 ['nodeidA7' 'BinGNN.py' 'funcname' 7
  'from torch_geometric import __version__ as tg_version\n']
 ['nodeidA8' 'BinGNN.py' 'funcname' 8
  'from torch_geometric.loader import DataLoader\n']
 ['nodeidA9' 'BinGNN.py' 'funcname' 9
  'from libs.dataset import GetDataset\n']
 ['nodeidA10' 'BinGNN.py' 'funcname' 10
  '# from libs.graphvis import VisualGraph, VisualGraphs\n']
 ['nodeidA11' 'BinGNN.py' 'funcname' 11
  'from libs.utils import TrainTestSplit, OutputEval, SaveBestModel, EndEpochLoop\n']
 ['nodeidA12' 'BinGNN.py' 'funcname' 12
  'from libs.nets.BGNN_sliced import BGNN, BGNNTrain, BGNNTest\n']
 ['nodeidA13' 'BinGNN.py' 'funcname' 13 '\n']
 ['nodeidA

In [39]:
# read graph of B.
lines = open('b/BinGNN.py', 'r').readlines()
nodesB = np.array([['nodeidB'+str(linenum+1), 'BinGNN.py', 'funcname', linenum+1, lines[linenum]] for linenum in range(len(lines))], dtype=object)
print(nodesB)

[['nodeidB1' 'BinGNN.py' 'funcname' 1 'import os\n']
 ['nodeidB2' 'BinGNN.py' 'funcname' 2 'import sys\n']
 ['nodeidB3' 'BinGNN.py' 'funcname' 3 'import time\n']
 ['nodeidB4' 'BinGNN.py' 'funcname' 4 'import argparse\n']
 ['nodeidB5' 'BinGNN.py' 'funcname' 5 'import numpy as np\n']
 ['nodeidB6' 'BinGNN.py' 'funcname' 6 'import torch\n']
 ['nodeidB7' 'BinGNN.py' 'funcname' 7
  'from torch_geometric import __version__ as tg_version\n']
 ['nodeidB8' 'BinGNN.py' 'funcname' 8
  'from torch_geometric.loader import DataLoader\n']
 ['nodeidB9' 'BinGNN.py' 'funcname' 9
  'from libs.dataset import GetDataset\n']
 ['nodeidB10' 'BinGNN.py' 'funcname' 10
  '# from libs.graphvis import VisualGraph, VisualGraphs\n']
 ['nodeidB11' 'BinGNN.py' 'funcname' 11
  'from libs.utils import TrainTestSplit, OutputEval, SaveBestModel, EndEpochLoop\n']
 ['nodeidB12' 'BinGNN.py' 'funcname' 12
  'from libs.nets.BGNN_sliced import BGNN, BGNNTrain, BGNNTest\n']
 ['nodeidB13' 'BinGNN.py' 'funcname' 13 '\n']
 ['nodeidB

In [40]:
# read the results in diff file.
def ReadDiff(filename):
    lines = open(filename, 'r').readlines()
    pattern = r'@@ -(\d+),*(\d+)* \+(\d+),*(\d+)* @@.*'
    atLines = []
    for line in lines:
        contents = re.findall(pattern, line)
        if len(contents):
            atLines.append([int(c) if len(c) else 1 for c in contents[0]])
    return np.array(atLines)

atLines = ReadDiff('p/BinGNN.py.patch')
print(atLines)

[[ 25   1  25   1]
 [ 51   1  51   1]
 [ 55   1  55   2]
 [ 65   1  66   6]
 [ 95   1 101   1]
 [100   1 106   1]
 [115   0 122   1]]


In [41]:
# re-order A
def ReOrder(nodes, atLines, version='A'):
    if version not in ['A', 'B']:
        print('[ERROR]')
        return nodes
    
    acc = 0
    for line in atLines:
        threshold = line[0] if version == 'A' else line[2]
        threshold += acc
        diff = line[3] if version == 'A' else line[1]
        acc += diff
        # print(threshold, diff, acc)
        for node in nodes:
            if node[-2] > threshold:
                node[-2] += diff
            elif node[-2] == threshold and version == 'B':
                node[-2] += diff
    return nodes

nodesA_new = ReOrder(nodesA, atLines, 'A')
# print(nodesA)
nodesB_new = ReOrder(nodesB, atLines, 'B')
# print(nodesB)

In [42]:
# test.
i, j = 0, 0
for linenum in range(130):
    lA, lB = '-', '-'
    codeA, codeB = '\n', '\n'
    if i < len(nodesA_new) and nodesA_new[i][-2] == linenum:
        lA = nodesA_new[i][-2]
        codeA = nodesA_new[i][-1]
        i += 1
    if j < len(nodesB_new) and nodesB_new[j][-2] == linenum:
        lB = nodesB_new[j][-2]
        codeB = nodesB_new[j][-1]
        j += 1
    print(linenum)
    print(lA, codeA, end='')
    print(lB, codeB, end='')
    print('-----------------------------')

0
- 
- 
-----------------------------
1
1 import os
1 import os
-----------------------------
2
2 import sys
2 import sys
-----------------------------
3
3 import time
3 import time
-----------------------------
4
4 import argparse
4 import argparse
-----------------------------
5
5 import numpy as np
5 import numpy as np
-----------------------------
6
6 import torch
6 import torch
-----------------------------
7
7 from torch_geometric import __version__ as tg_version
7 from torch_geometric import __version__ as tg_version
-----------------------------
8
8 from torch_geometric.loader import DataLoader
8 from torch_geometric.loader import DataLoader
-----------------------------
9
9 from libs.dataset import GetDataset
9 from libs.dataset import GetDataset
-----------------------------
10
10 # from libs.graphvis import VisualGraph, VisualGraphs
10 # from libs.graphvis import VisualGraph, VisualGraphs
-----------------------------
11
11 from libs.utils import TrainTestSplit, OutputEval, 